In [1]:
import sys
import copy
import random
import hashlib
import cupy as cp 
import numpy as np
from math import pi
import numpy.linalg as LA
from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('../..')))

from datetime import datetime
from os.path import expanduser

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from LevelSetPy.Grids import *
from LevelSetPy.Utilities import *
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration import *
from BRATVisualization.rcbrt_visu import RCBRTVisualizer

%matplotlib inline

In [2]:
args = Bundle(dict(visualize=True, flock_num=4, resume="murmurations_flock_01_02-06-22_17-43.hdf5", flock_payoff=False, pause_time=1e-5))

In [3]:
grid_min = expand(np.array((-.75, -1.25, -np.pi)), ax = 1)
grid_max = expand(np.array((3.25, 1.25, np.pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 100
N = np.array(([[
                resolution,
                np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                            (grid_max[0, 0] - grid_min[0, 0])),
                resolution-1
                ]])).T.astype(int)
grid_max[2, 0] *= (1-2/N[2,0])
g = createGrid(grid_min, grid_max, N, pdDims)

### Dynamics of the System

$x_1$ is horizontal range; $x_2$ is altitude; $x_3$ is horizontal velocity; $x_4$ is vertical velocity

$$
\begin{align}
    &\dot{x}_1 = x_3 \\
    &\dot{x}_2 = x_4 \\
    &\dot{x}_3 = a\cos(u) + d\cos(v) \\
    &\dot{x}_4 = a\sin(u) + d\sin(v) - g
\end{align}
$$
Write it as compact form
$$
\begin{align}
\dot{x} = f(x) + l(u) + g(v)
\end{align}
$$

In [8]:
def system(a, d, x, u, v):
    '''
    dx/dt = f(x) + l(u) + g(v) 
    input:
        a/d - acceleration of propulsive force/wind 
        x - state
        u - control input (angle of propulsive force)
        v - disturbance (angle of wind)
    output:
        dx - derivative of state
        f_x - derivative of f with respect to x
        l_u - derivative of l with respect to u
        g_v - derivative of l with respect to v
    '''
    g = 32.0 # gravitational acceleration, 32ft/sec^2
    
    dx = np.array([x[2], x[3], a*np.cos(u)+d*np.cos(v), a*np.sin(u)+d*np.sin(v)-g])
    f_x = np.array([[.0, .0, 1.0, .0],[.0, .0, .0, 1.0], [.0 , .0, .0, .0,], [.0 , .0, .0, .0,]])
    l_u = np.array([.0, .0, -a*np.sin(u), a*np.cos(u)])
    g_v = np.array([.0, .0, -d*np.sin(v), d*np.cos(v)])
    
    return dx, f_x, l_u, g_v

## Hamiltonian

$$H(x,u,v,p) = p_1x_3 + p_2x_4 + p_3(a\cos(u) + d\cos(v)) + p_4(a\sin(u)+d\sin(v)-g)$$
Partial derivative of $H$ with respect to $u$
$$
\begin{align}
&H_u = -p_3a\sin(u) + p_4a\cos(u) \\
&H_{uu} = -p_3a\cos(u) - p_4a\sin(u) \\
&H_{uv} = 0 \\
&H_{ux} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$
Partial derivative of $H$ with respect to $v$
$$
\begin{align}
&H_v = -p_3d\sin(v) + p_4d\cos(v) \\
&H_{vv} = -p_3d\cos(v) - p_4d\sin(v) \\
&H_{vu} = 0 \\
&H_{vx} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$

Partial derivative of $H$ with respect to $x$
$$
\begin{align}
&H_x = \begin{bmatrix}0 &0 &p_1 &p_2 \end{bmatrix}^T \\
&H_{xx} = 0_{4\times4} \\
&H_{xu} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T\\
&H_{xv} = \begin{bmatrix}0 &0 &0 &0 \end{bmatrix}^T
\end{align}
$$

In [9]:
def hamiltonian(a, b, x, u, v, p):
    '''
    input: 
        a/b - acceleration of propulsive force/wind         
        x - state
        u - control input
        v - disturbance
        p - costate
    output:
        H_u, H_ux, H_uu, H_v, H_vx, H_vv, H_uv, H_xx, H_xu, H_xv - derivative of Hammilton with respect to coresponding variables
    '''
    
    H_u = -p[2]*a*np.sin(u) + p[3]*a*np.cos(u)         
    H_uu = -p[2]*a*np.cos(u) - p[3]*a*np.sin(u)    
    H_uv = np.array([0]) 
    H_ux = np.array([.0, .0, .0, .0])
    
    H_v = -p[2]*d*np.sin(v) + p[3]*d*np.cos(v)      
    H_vv = -p[2]*d*np.cos(v) - p[3]*d*np.sin(v) 
    H_vx =  np.array([.0, .0, .0, .0])
    

    H_x = np.array([.0, .0, p[0], p[1]])
    H_xx = np.zeros([4, 4])

    return H_u, H_uu, H_uv, H_ux, H_v, H_vv, H_vx, H_x, H_xx
    

    

In [ ]:
def forward(x_r, u_opt, v_opt):
    '''
    input:
        x_r - trajectory of state at given time grid
        u_opt - trajectory of input at given time grid
        v_opt - trajectory of disturbance at given time grid
    output:
        V_opt - value function
        u_r_opt
        v_r_opt
    '''

    
    
    
    
    
    

In [ ]:
def backward(x_r, u_r, v_r, t_grid):
    '''
    input:
        x_r - trajectory of state at given time grid
        u_r - trajectory of input at given time grid
        v_r - trajectory of disturbance at given time grid
        t_grid - time grid partitions the time interval [-T, 0]
    output:
        V_r - value function at 
    '''
        

In [4]:
"""
    What we do here is really simple. We run as many 
    a fwd and backward pass of ddp algos as possible  
    on trajectories that are finely spaced apart by g.dx 
    
    This part accumulates all trajectories within grid bounds
"""
V_buf = []
all_traj = [copy.deepcopy(grid_min)]
while np.any(all_traj[-1]<grid_max):
    all_traj += [all_traj[-1]+g.dx]

all_traj = np.array(all_traj[:resolution]).squeeze()

# fix the inconsistencies in (x1, x2, x3)
indices = [np.nan for idx in range(len(all_traj))]
for dim in range(all_traj.shape[-1]):
    indices[dim] = all_traj[:,dim]>grid_max[dim,0]
    
# replace trajectories with bounds that exceed max 
# values along respective dimensions
for dim in range(all_traj.shape[-1]):
    all_traj[indices[dim],dim] = grid_max[dim,0]

In [ ]:
x = [1.5, 2.5, 3.0] 
dx = .1

grid_min = [1, 2, 2.5]
grid_max = [2, 3.5, 4.5]

X = [grid_min]
for i < N:
    X += X[-1]+dx

In [7]:
"""
    Once we have the trajectories, we can start with the 
    usual backward and forward pass of DDP.
    
    This will require two loops:
        Upper loop: Whereupon we iterate throuh every possible trajectory in 
        the system
        Lower loop: Forward and backward passes of DDP
"""
x = copy.copy(all_traj)
xr = 
# Here we go:
for traj_idx, traj in enumerate(all_traj):
    

SyntaxError: invalid syntax (<ipython-input-7-dff177466b76>, line 11)

In [39]:
grid_max.T, grid_max.shape

(array([[3.25      , 1.25      , 3.07812614]]), (3, 1))

### Polytope test

Given a matrix $A \in \mathbb{R}^n$ and a vector $b \in \mathbb{R}^m$, an $\mathcal{H}-$ polyhedron is

\begin{align}
    \mathcal{P} = \left\{x \in \mathbb{R}^n \mid Ax \le b \right\}
\end{align}

In [33]:
# x = np.linspace(0.1, 10, 3)

A = np.array([
     [.3, .7, 5.5],
     [8.4, 2.3, 6.7],
     [1.9, 2.5, 8.8]])

b = [1, 2, 3]


# Compute various x's
import numpy.linalg as LA 
Xes = []




array([[3.0000e-02, 3.5350e+00, 5.5000e+01],
       [8.4000e-01, 1.1615e+01, 6.7000e+01],
       [1.9000e-01, 1.2625e+01, 8.8000e+01]])